# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL") \
    .getOrCreate()
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)
user_log.createOrReplaceTempView("user_log_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [7]:
empty_string_set = set(spark.sql('''
          SELECT DISTINCT(page)
          FROM user_log_table 
          WHERE userID == ''
          '''
          ).collect())
nominal_set = set(spark.sql('''
          SELECT DISTINCT(page)
          FROM user_log_table
          '''
          ).collect())
nominal_set - empty_string_set

{Row(page='Downgrade'),
 Row(page='Error'),
 Row(page='Logout'),
 Row(page='NextSong'),
 Row(page='Save Settings'),
 Row(page='Settings'),
 Row(page='Submit Downgrade'),
 Row(page='Submit Upgrade'),
 Row(page='Upgrade')}

# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [21]:
spark.sql('''
          SELECT count(DISTINCT(userID)), gender
          FROM user_log_table
          GROUP BY gender
          '''
          ).collect()

[Row(count(DISTINCT userID)=462, gender='F'),
 Row(count(DISTINCT userID)=1, gender=None),
 Row(count(DISTINCT userID)=501, gender='M')]

In [23]:
spark.sql('''
          SELECT count(DISTINCT(userID))
          FROM user_log_table
          WHERE gender = 'F'
          '''
          ).collect()

[Row(count(DISTINCT userID)=462)]

# Question 4

How many songs were played from the most played artist?

In [24]:
spark.sql('''
          SELECT count(*), artist
          FROM user_log_table
          GROUP BY artist
          ORDER BY count(*) DESC
          '''
          ).show()

+--------+--------------------+
|count(1)|              artist|
+--------+--------------------+
|    1653|                null|
|      83|            Coldplay|
|      69|       Kings Of Leon|
|      52|Florence + The Ma...|
|      46|            BjÃÂ¶rk|
|      45|       Dwight Yoakam|
|      43|       Justin Bieber|
|      40|      The Black Keys|
|      37|         OneRepublic|
|      36|                Muse|
|      36|        Jack Johnson|
|      31|           Radiohead|
|      29|        Taylor Swift|
|      28|Barry Tuckwell/Ac...|
|      28|          Lily Allen|
|      28|               Train|
|      27|           Daft Punk|
|      27|           Metallica|
|      27|          Nickelback|
|      26|          Kanye West|
+--------+--------------------+
only showing top 20 rows



In [26]:
spark.sql('''
          SELECT count(*), artist
          FROM user_log_table
          WHERE artist IS NOT NULL
          GROUP BY artist
          ORDER BY count(*) DESC
          '''
          ).show()

+--------+--------------------+
|count(1)|              artist|
+--------+--------------------+
|      83|            Coldplay|
|      69|       Kings Of Leon|
|      52|Florence + The Ma...|
|      46|            BjÃÂ¶rk|
|      45|       Dwight Yoakam|
|      43|       Justin Bieber|
|      40|      The Black Keys|
|      37|         OneRepublic|
|      36|        Jack Johnson|
|      36|                Muse|
|      31|           Radiohead|
|      29|        Taylor Swift|
|      28|               Train|
|      28|Barry Tuckwell/Ac...|
|      28|          Lily Allen|
|      27|           Daft Punk|
|      27|           Metallica|
|      27|          Nickelback|
|      26|          Kanye West|
|      24|Red Hot Chili Pep...|
+--------+--------------------+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [30]:
home_results = spark.sql("""SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS 
                       page_status FROM user_log_table 
                       WHERE (page = 'NextSong') or (page = 'Home') 
                    """)

home_results.createOrReplaceTempView("home_results_table")

In [33]:
cumulative_sum = spark.sql("""SELECT *, SUM(page_status) OVER 
                            (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN 
                                    UNBOUNDED PRECEDING AND CURRENT ROW) AS period 
                              FROM home_results_table""")


In [34]:
cumulative_sum.show()

+------+--------+-------------+-----------+------+
|userID|    page|           ts|page_status|period|
+------+--------+-------------+-----------+------+
|  1436|NextSong|1513783259284|          0|     0|
|  1436|NextSong|1513782858284|          0|     0|
|  2088|    Home|1513805972284|          1|     1|
|  2088|NextSong|1513805859284|          0|     1|
|  2088|NextSong|1513805494284|          0|     1|
|  2088|NextSong|1513805065284|          0|     1|
|  2088|NextSong|1513804786284|          0|     1|
|  2088|NextSong|1513804555284|          0|     1|
|  2088|NextSong|1513804196284|          0|     1|
|  2088|NextSong|1513803967284|          0|     1|
|  2088|NextSong|1513803820284|          0|     1|
|  2088|NextSong|1513803651284|          0|     1|
|  2088|NextSong|1513803413284|          0|     1|
|  2088|NextSong|1513803254284|          0|     1|
|  2088|NextSong|1513803057284|          0|     1|
|  2088|NextSong|1513802824284|          0|     1|
|  2162|NextSong|1513781246284|

In [35]:
# keep the results in a view
cumulative_sum.createOrReplaceTempView("period_of_use")

In [36]:
# find the average count for NextSong
spark.sql("""SELECT AVG(count_results) FROM 
          (SELECT COUNT(*) AS count_results FROM period_of_use 
          GROUP BY userID, period, page HAVING page = 'NextSong') AS counts""").show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

